# Лабораторна робота 4.

[Посилання на GitHub](https://github.com/unIns1de/MN/tree/main)


1. Завантаження даних MovieLens 100k

url = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()  # розпаковує файли у папку ml-100k


In [11]:
# Завантажуємо дані
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id','item_id','rating','timestamp'])
movies = pd.read_csv('ml-100k/u.item', sep='|', encoding='latin-1', names=range(24))
movies = movies[[0,1]]
movies.columns = ['item_id','title']

2. Матриця рейтингів


In [12]:
R = ratings.pivot(index='user_id', columns='item_id', values='rating').fillna(0)

3. MNK-рекомендації (User-based) - ВИПРАВЛЕНА ВЕРСІЯ



In [13]:
user_similarity = cosine_similarity(R)
user_similarity_df = pd.DataFrame(user_similarity, index=R.index, columns=R.index)

def recommend_mnk(user_id, R, user_similarity_df, top_n=5):
    # Отримуємо схожість поточного користувача з іншими
    similar_users = user_similarity_df[user_id]

    # Видаляємо поточного користувача
    similar_users = similar_users.drop(user_id)

    # Отримуємо рейтинги схожих користувачів
    R_similar = R.loc[similar_users.index]

    # Обчислюємо зважені рейтинги
    weighted_ratings = (R_similar.T * similar_users.values).sum(axis=1) / similar_users.sum()
    weighted_ratings = pd.Series(weighted_ratings, index=R.columns)

    # Фільтруємо фільми, які користувач ще не оцінював
    user_rated = R.loc[user_id] > 0
    recommended_items = weighted_ratings[~user_rated].sort_values(ascending=False)

    return movies.set_index('item_id').loc[recommended_items.head(top_n).index]['title']

print("Рекомендації MNK для user_id=1:")
print(recommend_mnk(1, R, user_similarity_df))

Рекомендації MNK для user_id=1:
item_id
318                   Schindler's List (1993)
423         E.T. the Extra-Terrestrial (1982)
357    One Flew Over the Cuckoo's Nest (1975)
286               English Patient, The (1996)
288                             Scream (1996)
Name: title, dtype: object


4. Matrix Factorization (градієнтний спуск) - ВИПРАВЛЕНА ВЕРСІЯ


In [14]:

class MF:
    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R.values
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        self.P = np.random.normal(0, 0.1, (self.num_users, self.K))
        self.Q = np.random.normal(0, 0.1, (self.num_items, self.K))

        for _ in range(self.iterations):
            for i in range(self.num_users):
                for j in range(self.num_items):
                    if self.R[i, j] > 0:
                        eij = self.R[i, j] - np.dot(self.P[i, :], self.Q[j, :].T)
                        self.P[i, :] += self.alpha * (2 * eij * self.Q[j, :] - self.beta * self.P[i, :])
                        self.Q[j, :] += self.alpha * (2 * eij * self.P[i, :] - self.beta * self.Q[j, :])
        return self

    def predict(self):
        return pd.DataFrame(np.dot(self.P, self.Q.T), index=R.index, columns=R.columns)

mf_model = MF(R, K=10, alpha=0.002, beta=0.02, iterations=20).train()
R_pred = mf_model.predict()

# Рекомендації для користувача 1
user_rated = R.loc[1] > 0
mf_recs = R_pred.loc[1][~user_rated].sort_values(ascending=False)

print("\nРекомендації MF для user_id=1:")
print(movies.set_index('item_id').loc[mf_recs.head(5).index]['title'])


Рекомендації MF для user_id=1:
item_id
408                                Close Shave, A (1995)
480                            North by Northwest (1959)
357               One Flew Over the Cuckoo's Nest (1975)
483                                    Casablanca (1942)
474    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object


5. Кластеризація користувачів - ВИПРАВЛЕНА ВЕРСІЯ


In [15]:

kmeans = KMeans(n_clusters=5, random_state=42)
cluster_labels = kmeans.fit_predict(R)
R_clustered = R.copy()
R_clustered['cluster'] = cluster_labels
user_cluster = R_clustered.loc[1, 'cluster']

# Знаходимо середні рейтинги в кластері
cluster_users = R_clustered[R_clustered['cluster'] == user_cluster].drop('cluster', axis=1)
cluster_mean = cluster_users.mean()

# Фільтруємо фільми, які користувач ще не бачив
user_rated = R.loc[1] > 0
cluster_recs = cluster_mean[~user_rated].sort_values(ascending=False)

print("\nРекомендації кластеризації для user_id=1:")
print(movies.set_index('item_id').loc[cluster_recs.head(5).index]['title'])


Рекомендації кластеризації для user_id=1:
item_id
423    E.T. the Extra-Terrestrial (1982)
568                         Speed (1994)
385                     True Lies (1994)
405           Mission: Impossible (1996)
403                        Batman (1989)
Name: title, dtype: object
